Milan San-Remo é uma das corridas mais antigas de bicicleta , uma das suas principais caracteristicas é ser imprevisivel, qualquer tipo de ciclista tem a chance de obter uma vitoria nessa clássica corrida italia. Neste projeto irei analisar os principais campeões do século XXI, levando informações como idade, peso, nacionalidade e tipo de ciclista.

Primeira etapa:
Web Scramping, a primeiro etapa do projeto é coletar os nomes dos campeões da prova e suas principais caracteristicas (idade, peso, nacionalidade) importante ressaltar que essas informações podem ter oscilações com o passar dos anos, utilizaremos informações do site Wikipédia, 

In [48]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd
from datetime import datetime

# URL da página que será feito o web scraping
url = "https://en.wikipedia.org/wiki/Milan%E2%80%93San_Remo"

# Faz a requisição HTTP para obter o conteúdo da página
response = requests.get(url)

# Cria um objeto BeautifulSoup com o conteúdo HTML da página
soup = BeautifulSoup(response.content, "html.parser")

# Encontra a tabela de vencedores com base na classe CSS da tabela
table = soup.find("table", {"class": "wikitable"})

# Extrai os dados da tabela em formato de lista
table_data = []
rows = table.find_all("tr")
for row in rows:
    cells = row.find_all(["th", "td"])
    row_data = [cell.get_text(strip=True) for cell in cells]
    if row_data:
        table_data.append(row_data)

# Filtra apenas os vencedores do século XXI
table_data = [row for row in table_data if row[0].isdigit() and int(row[0]) >= 2001]

# Lista para armazenar os dados dos atletas
athlete_data = []

# Percorre as linhas da tabela
for row in table_data:
    # Obtém o link do nome do atleta e trata corretamente o link
    athlete_link = urljoin(url, row[2])
    
    # Faz a requisição HTTP para obter o conteúdo da página do atleta
    athlete_response = requests.get(athlete_link)
    
    # Cria um objeto BeautifulSoup com o conteúdo HTML da página do atleta
    athlete_soup = BeautifulSoup(athlete_response.content, "html.parser")
    
    # Encontra a tabela de informações pessoais do atleta
    infobox = athlete_soup.find("table", {"class": "infobox vcard"})
    
    # Extrai as informações de altura, peso e data de nascimento (se disponíveis)
    height_text = infobox.find("th", text="Height").find_next_sibling("td").get_text(strip=True)
    weight_text = infobox.find("th", text="Weight").find_next_sibling("td").get_text(strip=True)
    birth_date = infobox.find("span", {"class": "bday"}).get_text(strip=True)
    
    # Limpa os dados de altura e peso para obter apenas os valores numéricos
    height = height_text.split()[0]
    weight = weight_text.split()[0]
    
    # Converte altura e peso para o tipo de dado apropriado
    height = float(height.replace(',', '.'))
    weight = int(weight)
    
    # Obtém o ano de vitória, equipe e país do atleta
    year = int(row[0])
    team = row[3]
    country = row[1]
    
    # Calcula a idade do atleta quando venceu
    birth_date_obj = datetime.strptime(birth_date, "%Y-%m-%d")
    age = year - birth_date_obj.year
    
    # Armazena os dados do atleta em um dicionário
    athlete_info = {
        "Ano": year,
        "Nome": row[2], 
        "Altura": height,
        "Peso": weight,
        "Ano de Nascimento": birth_date,
        "Idade": age,
        "Time": team,
        "País": country
        
    }
    
    # Adiciona o dicionário à lista de dados dos atletas
    athlete_data.append(athlete_info)

# Cria um DataFrame com os dados dos atletas
lista_vencedores = pd.DataFrame(athlete_data, columns=["Ano", "Nome",  "Altura", "Peso", "Idade", "Ano de Nascimento", "País","Time"])



In [49]:
lista_vencedores.head(24)

,Ano,Nome,Altura,Peso,Idade,Ano de Nascimento,País,Time
0,2001,Erik Zabel,1.76,69,31,1970-07-07,Germany,Team Telekom
1,2002,Mario Cipollini,1.89,76,35,1967-03-22,Italy,Acqua e Sapone–Cantina Tollo
2,2003,Paolo Bettini,1.69,58,29,1974-04-01,Italy,Quick-Step–Davitamon
3,2004,Óscar Freire,1.71,64,28,1976-02-15,Spain,Rabobank
4,2005,Alessandro Petacchi,1.84,75,31,1974-01-03,Italy,Fassa Bortolo
5,2006,Filippo Pozzato,1.83,73,25,1981-09-10,Italy,Quick-Step–Innergetic
6,2007,Óscar Freire,1.71,64,31,1976-02-15,Spain,Rabobank
7,2008,Fabian Cancellara,1.86,82,27,1981-03-18,Switzerland,Team CSC
8,2009,Mark Cavendish,1.75,70,24,1985-05-21,Great Britain,Team Columbia–High Road
9,2010,Óscar Freire,1.71,64,34,1976-02-15,Spain,Rabobank


Segunda etapa:
Com o data frame criado, iremos salvar a tabela em formato excel na máquina, para evitar que alterações no site prejudique análises futuras, e facilitar a importação da tabela para outros programas

In [50]:
#Criando uma cópia em formato excel na máquina

!pip install openpyxl

caminho_arquivo = 'C:\\Users\\diego\\OneDrive\\Documentos\\Projeto_Python\\Milan San Remo\\lista_vencedores_milan_san_remo.xlsx'

lista_vencedores.to_excel(caminho_arquivo, index=False)


Terceira etapa:
Criando o "ciclista ideal" para a Milan San-Remo, com análises estátiscas dos ultimos campeões

In [52]:
#Utilizando média e moda

altura_media = lista_vencedores['Altura'].mean()
peso_medio = lista_vencedores['Peso'].mean()
idade_media = lista_vencedores['Idade'].mean() 
pais_moda = lista_vencedores['País'].mode()

print(f'A persona do campeão da milan San-Remo teria as seguintes caracateristicas: ')
print(f'Altura: {altura_media:.2f}')
print(f'Peso: {peso_medio:.0f}')
print(f'Idade: {idade_media:.0f}')
print(f'País: {pais_moda}')

A persona do campeão da milan San-Remo teria as seguintes caracateristicas: 
Altura: 1.79
Peso: 71
Idade: 29
País: 0    Italy
Name: País, dtype: object
